In [1]:
import os, random
from pydub import AudioSegment
from xml.dom import minidom
import numpy as np
import pandas as pd
from pysndfx import AudioEffectsChain
from librosa import load
import shutil

In [10]:
f = open("D://Background_noise//meta.txt","r")
contents = f.read()
match_noise_to_type = []
for item in contents.split('	'):
    y = item.split('\naudio/')
    try:
        match_noise_to_type.append((y[1], y[0]))
    except:
        pass

noise_dict = dict(match_noise_to_type)

{'a001_120_150.wav': 'residential_area',
 'a001_150_180.wav': 'residential_area',
 'a001_30_60.wav': 'residential_area',
 'a001_60_90.wav': 'residential_area',
 'a001_90_120.wav': 'residential_area',
 'a003_0_30.wav': 'residential_area',
 'a003_120_150.wav': 'residential_area',
 'a003_150_180.wav': 'residential_area',
 'a003_180_210.wav': 'residential_area',
 'a003_210_240.wav': 'residential_area',
 'a003_30_60.wav': 'residential_area',
 'a003_60_90.wav': 'residential_area',
 'a003_90_120.wav': 'residential_area',
 'a005_0_30.wav': 'residential_area',
 'a005_150_180.wav': 'city_center',
 'a005_210_240.wav': 'city_center',
 'a005_30_60.wav': 'city_center',
 'a005_60_90.wav': 'city_center',
 'a005_90_120.wav': 'city_center',
 'a006_0_30.wav': 'city_center',
 'a006_150_180.wav': 'beach',
 'a006_30_60.wav': 'beach',
 'a006_60_90.wav': 'beach',
 'a006_90_120.wav': 'beach',
 'a009_0_30.wav': 'beach',
 'a009_120_150.wav': 'residential_area',
 'a009_150_180.wav': 'residential_area',
 'a009_180

In [33]:
h_directory = 'D://Datasets//TRAINING//Happy//'
a_directory = 'D://Datasets//TRAINING//Angry//'
n_directory = 'D://Datasets//TRAINING//Neutral//'
s_directory = 'D://Datasets//TRAINING//Sad//'
o_directory = 'D://Datasets//TRAINING//Other//'

h_test = 'D://Datasets//TRAINING//Happy_test//'
a_test = 'D://Datasets//TRAINING//Angry_test//'
n_test = 'D://Datasets//TRAINING//Neutral_test//'
s_test = 'D://Datasets//TRAINING//Sad_test//'
o_test = 'D://Datasets//TRAINING//Other_test//'

h_EMO = 'D://Datasets//EMO-DB//wav//Happy//'
a_EMO = 'D://Datasets//EMO-DB//wav//Angry//'
n_EMO = 'D://Datasets//EMO-DB//wav//Neutral//'
s_EMO = 'D://Datasets//EMO-DB//wav//Sad//'
o_EMO = 'D://Datasets//EMO-DB//wav//Other//'



background_noise = 'D://Background_noise//TUT2016//'
background_noise_test = 'D://Background_noise//noise_test//'

emotion_train_folders = [h_directory, a_directory, n_directory, s_directory, o_directory]
emotion_test_folders = [h_test, a_test, n_test, s_test, o_test]

In [47]:
def rename_emotion_wavs(emotion_directory):
    if 'Happy' in emotion_directory:
        word = 'Happy'
    elif 'Angry' in emotion_directory:
        word = 'Angry'
    elif 'Neutral' in emotion_directory:
        word = 'Neutral'
    elif 'Sad' in emotion_directory:
        word = 'Sad'
    else:
        word = 'Other'
    
    index = 0
    for file in os.listdir(emotion_directory):
        src = emotion_directory + file
        dst = emotion_directory + word + str(index) + '.wav'
        try:
            os.rename(src, dst)
        except WindowsError:
            os.remove(dst)
            os.rename(src, dst)
        index += 1
        print(dst)
        
#rename_emotion_wavs(h_directory)
#rename_emotion_wavs(a_directory)
#rename_emotion_wavs(n_directory)
#rename_emotion_wavs(s_directory)
#rename_emotion_wavs(o_directory)

D://Datasets//TRAINING//Happy//Happy0.wav
D://Datasets//TRAINING//Happy//Happy1.wav
D://Datasets//TRAINING//Happy//Happy2.wav
D://Datasets//TRAINING//Happy//Happy3.wav
D://Datasets//TRAINING//Happy//Happy4.wav
D://Datasets//TRAINING//Happy//Happy5.wav
D://Datasets//TRAINING//Happy//Happy6.wav
D://Datasets//TRAINING//Happy//Happy7.wav
D://Datasets//TRAINING//Happy//Happy8.wav
D://Datasets//TRAINING//Happy//Happy9.wav
D://Datasets//TRAINING//Happy//Happy10.wav
D://Datasets//TRAINING//Happy//Happy11.wav
D://Datasets//TRAINING//Happy//Happy12.wav
D://Datasets//TRAINING//Happy//Happy13.wav
D://Datasets//TRAINING//Happy//Happy14.wav
D://Datasets//TRAINING//Happy//Happy15.wav
D://Datasets//TRAINING//Happy//Happy16.wav
D://Datasets//TRAINING//Happy//Happy17.wav
D://Datasets//TRAINING//Happy//Happy18.wav
D://Datasets//TRAINING//Happy//Happy19.wav
D://Datasets//TRAINING//Happy//Happy20.wav
D://Datasets//TRAINING//Happy//Happy21.wav
D://Datasets//TRAINING//Happy//Happy22.wav
D://Datasets//TRAININ

D://Datasets//TRAINING//Happy//Happy193.wav
D://Datasets//TRAINING//Happy//Happy194.wav
D://Datasets//TRAINING//Happy//Happy195.wav
D://Datasets//TRAINING//Happy//Happy196.wav
D://Datasets//TRAINING//Happy//Happy197.wav
D://Datasets//TRAINING//Happy//Happy198.wav
D://Datasets//TRAINING//Happy//Happy199.wav
D://Datasets//TRAINING//Happy//Happy200.wav
D://Datasets//TRAINING//Happy//Happy201.wav
D://Datasets//TRAINING//Happy//Happy202.wav
D://Datasets//TRAINING//Happy//Happy203.wav
D://Datasets//TRAINING//Happy//Happy204.wav
D://Datasets//TRAINING//Happy//Happy205.wav
D://Datasets//TRAINING//Happy//Happy206.wav
D://Datasets//TRAINING//Happy//Happy207.wav
D://Datasets//TRAINING//Happy//Happy208.wav
D://Datasets//TRAINING//Happy//Happy209.wav
D://Datasets//TRAINING//Happy//Happy210.wav
D://Datasets//TRAINING//Happy//Happy211.wav
D://Datasets//TRAINING//Happy//Happy212.wav
D://Datasets//TRAINING//Happy//Happy213.wav
D://Datasets//TRAINING//Happy//Happy214.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy383.wav
D://Datasets//TRAINING//Happy//Happy384.wav
D://Datasets//TRAINING//Happy//Happy385.wav
D://Datasets//TRAINING//Happy//Happy386.wav
D://Datasets//TRAINING//Happy//Happy387.wav
D://Datasets//TRAINING//Happy//Happy388.wav
D://Datasets//TRAINING//Happy//Happy389.wav
D://Datasets//TRAINING//Happy//Happy390.wav
D://Datasets//TRAINING//Happy//Happy391.wav
D://Datasets//TRAINING//Happy//Happy392.wav
D://Datasets//TRAINING//Happy//Happy393.wav
D://Datasets//TRAINING//Happy//Happy394.wav
D://Datasets//TRAINING//Happy//Happy395.wav
D://Datasets//TRAINING//Happy//Happy396.wav
D://Datasets//TRAINING//Happy//Happy397.wav
D://Datasets//TRAINING//Happy//Happy398.wav
D://Datasets//TRAINING//Happy//Happy399.wav
D://Datasets//TRAINING//Happy//Happy400.wav
D://Datasets//TRAINING//Happy//Happy401.wav
D://Datasets//TRAINING//Happy//Happy402.wav
D://Datasets//TRAINING//Happy//Happy403.wav
D://Datasets//TRAINING//Happy//Happy404.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy570.wav
D://Datasets//TRAINING//Happy//Happy571.wav
D://Datasets//TRAINING//Happy//Happy572.wav
D://Datasets//TRAINING//Happy//Happy573.wav
D://Datasets//TRAINING//Happy//Happy574.wav
D://Datasets//TRAINING//Happy//Happy575.wav
D://Datasets//TRAINING//Happy//Happy576.wav
D://Datasets//TRAINING//Happy//Happy577.wav
D://Datasets//TRAINING//Happy//Happy578.wav
D://Datasets//TRAINING//Happy//Happy579.wav
D://Datasets//TRAINING//Happy//Happy580.wav
D://Datasets//TRAINING//Happy//Happy581.wav
D://Datasets//TRAINING//Happy//Happy582.wav
D://Datasets//TRAINING//Happy//Happy583.wav
D://Datasets//TRAINING//Happy//Happy584.wav
D://Datasets//TRAINING//Happy//Happy585.wav
D://Datasets//TRAINING//Happy//Happy586.wav
D://Datasets//TRAINING//Happy//Happy587.wav
D://Datasets//TRAINING//Happy//Happy588.wav
D://Datasets//TRAINING//Happy//Happy589.wav
D://Datasets//TRAINING//Happy//Happy590.wav
D://Datasets//TRAINING//Happy//Happy591.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy762.wav
D://Datasets//TRAINING//Happy//Happy763.wav
D://Datasets//TRAINING//Happy//Happy764.wav
D://Datasets//TRAINING//Happy//Happy765.wav
D://Datasets//TRAINING//Happy//Happy766.wav
D://Datasets//TRAINING//Happy//Happy767.wav
D://Datasets//TRAINING//Happy//Happy768.wav
D://Datasets//TRAINING//Happy//Happy769.wav
D://Datasets//TRAINING//Happy//Happy770.wav
D://Datasets//TRAINING//Happy//Happy771.wav
D://Datasets//TRAINING//Happy//Happy772.wav
D://Datasets//TRAINING//Happy//Happy773.wav
D://Datasets//TRAINING//Happy//Happy774.wav
D://Datasets//TRAINING//Happy//Happy775.wav
D://Datasets//TRAINING//Happy//Happy776.wav
D://Datasets//TRAINING//Happy//Happy777.wav
D://Datasets//TRAINING//Happy//Happy778.wav
D://Datasets//TRAINING//Happy//Happy779.wav
D://Datasets//TRAINING//Happy//Happy780.wav
D://Datasets//TRAINING//Happy//Happy781.wav
D://Datasets//TRAINING//Happy//Happy782.wav
D://Datasets//TRAINING//Happy//Happy783.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy952.wav
D://Datasets//TRAINING//Happy//Happy953.wav
D://Datasets//TRAINING//Happy//Happy954.wav
D://Datasets//TRAINING//Happy//Happy955.wav
D://Datasets//TRAINING//Happy//Happy956.wav
D://Datasets//TRAINING//Happy//Happy957.wav
D://Datasets//TRAINING//Happy//Happy958.wav
D://Datasets//TRAINING//Happy//Happy959.wav
D://Datasets//TRAINING//Happy//Happy960.wav
D://Datasets//TRAINING//Happy//Happy961.wav
D://Datasets//TRAINING//Happy//Happy962.wav
D://Datasets//TRAINING//Happy//Happy963.wav
D://Datasets//TRAINING//Happy//Happy964.wav
D://Datasets//TRAINING//Happy//Happy965.wav
D://Datasets//TRAINING//Happy//Happy966.wav
D://Datasets//TRAINING//Happy//Happy967.wav
D://Datasets//TRAINING//Happy//Happy968.wav
D://Datasets//TRAINING//Happy//Happy969.wav
D://Datasets//TRAINING//Happy//Happy970.wav
D://Datasets//TRAINING//Happy//Happy971.wav
D://Datasets//TRAINING//Happy//Happy972.wav
D://Datasets//TRAINING//Happy//Happy973.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy1142.wav
D://Datasets//TRAINING//Happy//Happy1143.wav
D://Datasets//TRAINING//Happy//Happy1144.wav
D://Datasets//TRAINING//Happy//Happy1145.wav
D://Datasets//TRAINING//Happy//Happy1146.wav
D://Datasets//TRAINING//Happy//Happy1147.wav
D://Datasets//TRAINING//Happy//Happy1148.wav
D://Datasets//TRAINING//Happy//Happy1149.wav
D://Datasets//TRAINING//Happy//Happy1150.wav
D://Datasets//TRAINING//Happy//Happy1151.wav
D://Datasets//TRAINING//Happy//Happy1152.wav
D://Datasets//TRAINING//Happy//Happy1153.wav
D://Datasets//TRAINING//Happy//Happy1154.wav
D://Datasets//TRAINING//Happy//Happy1155.wav
D://Datasets//TRAINING//Happy//Happy1156.wav
D://Datasets//TRAINING//Happy//Happy1157.wav
D://Datasets//TRAINING//Happy//Happy1158.wav
D://Datasets//TRAINING//Happy//Happy1159.wav
D://Datasets//TRAINING//Happy//Happy1160.wav
D://Datasets//TRAINING//Happy//Happy1161.wav
D://Datasets//TRAINING//Happy//Happy1162.wav
D://Datasets//TRAINING//Happy//Happy1163.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy1326.wav
D://Datasets//TRAINING//Happy//Happy1327.wav
D://Datasets//TRAINING//Happy//Happy1328.wav
D://Datasets//TRAINING//Happy//Happy1329.wav
D://Datasets//TRAINING//Happy//Happy1330.wav
D://Datasets//TRAINING//Happy//Happy1331.wav
D://Datasets//TRAINING//Happy//Happy1332.wav
D://Datasets//TRAINING//Happy//Happy1333.wav
D://Datasets//TRAINING//Happy//Happy1334.wav
D://Datasets//TRAINING//Happy//Happy1335.wav
D://Datasets//TRAINING//Happy//Happy1336.wav
D://Datasets//TRAINING//Happy//Happy1337.wav
D://Datasets//TRAINING//Happy//Happy1338.wav
D://Datasets//TRAINING//Happy//Happy1339.wav
D://Datasets//TRAINING//Happy//Happy1340.wav
D://Datasets//TRAINING//Happy//Happy1341.wav
D://Datasets//TRAINING//Happy//Happy1342.wav
D://Datasets//TRAINING//Happy//Happy1343.wav
D://Datasets//TRAINING//Happy//Happy1344.wav
D://Datasets//TRAINING//Happy//Happy1345.wav
D://Datasets//TRAINING//Happy//Happy1346.wav
D://Datasets//TRAINING//Happy//Happy1347.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy1513.wav
D://Datasets//TRAINING//Happy//Happy1514.wav
D://Datasets//TRAINING//Happy//Happy1515.wav
D://Datasets//TRAINING//Happy//Happy1516.wav
D://Datasets//TRAINING//Happy//Happy1517.wav
D://Datasets//TRAINING//Happy//Happy1518.wav
D://Datasets//TRAINING//Happy//Happy1519.wav
D://Datasets//TRAINING//Happy//Happy1520.wav
D://Datasets//TRAINING//Happy//Happy1521.wav
D://Datasets//TRAINING//Happy//Happy1522.wav
D://Datasets//TRAINING//Happy//Happy1523.wav
D://Datasets//TRAINING//Happy//Happy1524.wav
D://Datasets//TRAINING//Happy//Happy1525.wav
D://Datasets//TRAINING//Happy//Happy1526.wav
D://Datasets//TRAINING//Happy//Happy1527.wav
D://Datasets//TRAINING//Happy//Happy1528.wav
D://Datasets//TRAINING//Happy//Happy1529.wav
D://Datasets//TRAINING//Happy//Happy1530.wav
D://Datasets//TRAINING//Happy//Happy1531.wav
D://Datasets//TRAINING//Happy//Happy1532.wav
D://Datasets//TRAINING//Happy//Happy1533.wav
D://Datasets//TRAINING//Happy//Happy1534.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy1696.wav
D://Datasets//TRAINING//Happy//Happy1697.wav
D://Datasets//TRAINING//Happy//Happy1698.wav
D://Datasets//TRAINING//Happy//Happy1699.wav
D://Datasets//TRAINING//Happy//Happy1700.wav
D://Datasets//TRAINING//Happy//Happy1701.wav
D://Datasets//TRAINING//Happy//Happy1702.wav
D://Datasets//TRAINING//Happy//Happy1703.wav
D://Datasets//TRAINING//Happy//Happy1704.wav
D://Datasets//TRAINING//Happy//Happy1705.wav
D://Datasets//TRAINING//Happy//Happy1706.wav
D://Datasets//TRAINING//Happy//Happy1707.wav
D://Datasets//TRAINING//Happy//Happy1708.wav
D://Datasets//TRAINING//Happy//Happy1709.wav
D://Datasets//TRAINING//Happy//Happy1710.wav
D://Datasets//TRAINING//Happy//Happy1711.wav
D://Datasets//TRAINING//Happy//Happy1712.wav
D://Datasets//TRAINING//Happy//Happy1713.wav
D://Datasets//TRAINING//Happy//Happy1714.wav
D://Datasets//TRAINING//Happy//Happy1715.wav
D://Datasets//TRAINING//Happy//Happy1716.wav
D://Datasets//TRAINING//Happy//Happy1717.wav
D://Datase

In [22]:
def rename_noise_wavs(noise_directory):
    index = 0
    for noise in os.listdir(noise_directory):
        try:
            noise_type = noise_dict.get(noise)
            new_noise_name = noise_type + '_' + str(index)
            src = noise_directory + noise
            dst = noise_directory + new_noise_name + '.wav'
            os.rename(src, dst) 
            index += 1
            
            print(dst)
        except:
            pass

# rename_noise_wavs(background_noise_test)
# rename_noise_wavs(background_noise)

D://Background_noise//noise_test//residential_area_0.wav
D://Background_noise//noise_test//residential_area_1.wav
D://Background_noise//noise_test//residential_area_2.wav
D://Background_noise//noise_test//residential_area_3.wav
D://Background_noise//noise_test//residential_area_4.wav
D://Background_noise//noise_test//residential_area_5.wav
D://Background_noise//noise_test//residential_area_6.wav
D://Background_noise//noise_test//residential_area_7.wav
D://Background_noise//noise_test//residential_area_8.wav
D://Background_noise//noise_test//beach_9.wav
D://Background_noise//noise_test//beach_10.wav
D://Background_noise//noise_test//beach_11.wav
D://Background_noise//noise_test//beach_12.wav
D://Background_noise//noise_test//park_13.wav
D://Background_noise//noise_test//park_14.wav
D://Background_noise//noise_test//park_15.wav
D://Background_noise//noise_test//park_16.wav
D://Background_noise//noise_test//park_17.wav
D://Background_noise//noise_test//park_18.wav
D://Background_noise//no

D://Background_noise//noise_test//grocery_store_171.wav
D://Background_noise//noise_test//grocery_store_172.wav
D://Background_noise//noise_test//grocery_store_173.wav
D://Background_noise//noise_test//car_174.wav
D://Background_noise//noise_test//city_center_175.wav
D://Background_noise//noise_test//city_center_176.wav
D://Background_noise//noise_test//city_center_177.wav
D://Background_noise//noise_test//city_center_178.wav
D://Background_noise//noise_test//city_center_179.wav
D://Background_noise//noise_test//car_180.wav
D://Background_noise//noise_test//car_181.wav
D://Background_noise//noise_test//car_182.wav
D://Background_noise//noise_test//car_183.wav
D://Background_noise//noise_test//city_center_184.wav
D://Background_noise//noise_test//city_center_185.wav
D://Background_noise//noise_test//city_center_186.wav
D://Background_noise//noise_test//city_center_187.wav
D://Background_noise//noise_test//grocery_store_188.wav
D://Background_noise//noise_test//bus_189.wav
D://Background

D://Background_noise//TUT2016//forest_path_156.wav
D://Background_noise//TUT2016//forest_path_157.wav
D://Background_noise//TUT2016//forest_path_158.wav
D://Background_noise//TUT2016//forest_path_159.wav
D://Background_noise//TUT2016//forest_path_160.wav
D://Background_noise//TUT2016//forest_path_161.wav
D://Background_noise//TUT2016//forest_path_162.wav
D://Background_noise//TUT2016//forest_path_163.wav
D://Background_noise//TUT2016//forest_path_164.wav
D://Background_noise//TUT2016//car_165.wav
D://Background_noise//TUT2016//car_166.wav
D://Background_noise//TUT2016//car_167.wav
D://Background_noise//TUT2016//car_168.wav
D://Background_noise//TUT2016//car_169.wav
D://Background_noise//TUT2016//car_170.wav
D://Background_noise//TUT2016//forest_path_171.wav
D://Background_noise//TUT2016//forest_path_172.wav
D://Background_noise//TUT2016//forest_path_173.wav
D://Background_noise//TUT2016//forest_path_174.wav
D://Background_noise//TUT2016//forest_path_175.wav
D://Background_noise//TUT201

D://Background_noise//TUT2016//city_center_339.wav
D://Background_noise//TUT2016//city_center_340.wav
D://Background_noise//TUT2016//car_341.wav
D://Background_noise//TUT2016//car_342.wav
D://Background_noise//TUT2016//car_343.wav
D://Background_noise//TUT2016//car_344.wav
D://Background_noise//TUT2016//car_345.wav
D://Background_noise//TUT2016//car_346.wav
D://Background_noise//TUT2016//car_347.wav
D://Background_noise//TUT2016//car_348.wav
D://Background_noise//TUT2016//car_349.wav
D://Background_noise//TUT2016//car_350.wav
D://Background_noise//TUT2016//car_351.wav
D://Background_noise//TUT2016//car_352.wav
D://Background_noise//TUT2016//bus_353.wav
D://Background_noise//TUT2016//bus_354.wav
D://Background_noise//TUT2016//bus_355.wav
D://Background_noise//TUT2016//bus_356.wav
D://Background_noise//TUT2016//bus_357.wav
D://Background_noise//TUT2016//bus_358.wav
D://Background_noise//TUT2016//bus_359.wav
D://Background_noise//TUT2016//bus_360.wav
D://Background_noise//TUT2016//train_3

D://Background_noise//TUT2016//beach_516.wav
D://Background_noise//TUT2016//beach_517.wav
D://Background_noise//TUT2016//beach_518.wav
D://Background_noise//TUT2016//beach_519.wav
D://Background_noise//TUT2016//beach_520.wav
D://Background_noise//TUT2016//beach_521.wav
D://Background_noise//TUT2016//beach_522.wav
D://Background_noise//TUT2016//beach_523.wav
D://Background_noise//TUT2016//park_524.wav
D://Background_noise//TUT2016//park_525.wav
D://Background_noise//TUT2016//park_526.wav
D://Background_noise//TUT2016//park_527.wav
D://Background_noise//TUT2016//park_528.wav
D://Background_noise//TUT2016//library_529.wav
D://Background_noise//TUT2016//library_530.wav
D://Background_noise//TUT2016//library_531.wav
D://Background_noise//TUT2016//library_532.wav
D://Background_noise//TUT2016//library_533.wav
D://Background_noise//TUT2016//home_534.wav
D://Background_noise//TUT2016//home_535.wav
D://Background_noise//TUT2016//home_536.wav
D://Background_noise//TUT2016//home_537.wav
D://Backg

D://Background_noise//TUT2016//metro_station_704.wav
D://Background_noise//TUT2016//metro_station_705.wav
D://Background_noise//TUT2016//metro_station_706.wav
D://Background_noise//TUT2016//metro_station_707.wav
D://Background_noise//TUT2016//metro_station_708.wav
D://Background_noise//TUT2016//metro_station_709.wav
D://Background_noise//TUT2016//metro_station_710.wav
D://Background_noise//TUT2016//metro_station_711.wav
D://Background_noise//TUT2016//metro_station_712.wav
D://Background_noise//TUT2016//metro_station_713.wav
D://Background_noise//TUT2016//metro_station_714.wav
D://Background_noise//TUT2016//metro_station_715.wav
D://Background_noise//TUT2016//metro_station_716.wav
D://Background_noise//TUT2016//metro_station_717.wav
D://Background_noise//TUT2016//metro_station_718.wav
D://Background_noise//TUT2016//metro_station_719.wav
D://Background_noise//TUT2016//metro_station_720.wav
D://Background_noise//TUT2016//tram_721.wav
D://Background_noise//TUT2016//tram_722.wav
D://Backgro

D://Background_noise//TUT2016//office_888.wav
D://Background_noise//TUT2016//office_889.wav
D://Background_noise//TUT2016//office_890.wav
D://Background_noise//TUT2016//office_891.wav


In [30]:
def delete_directory(path):
    shutil.rmtree(path, ignore_errors=True)

In [31]:
def partition_directory(path, new_path, percent):
    
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    else:
        delete_directory(new_path)
        os.makdir(new_path)
        
    total = len(os.listdir(path))
    test_number = int(total * percent)
    test_set = np.random.permutation(total)[0:test_number]
    
    for index in test_set:
        try:
            if os.listdir(path)[index].endswith('.wav'):
                original_location = path + os.listdir(path)[index]
                new_location = new_path + os.listdir(path)[index]
                os.rename(original_location, new_location)
                print(new_location)
        except:
            pass

    
    print('# of training data + # of testing data = ' + str(total))
    print(str(percent) + ' of the data is in training set')
    print('# of training data = ' + str(test_set.shape))


In [48]:
percent = 0.2
partition_directory(h_directory, h_test, percent)
#partition_directory(a_directory, a_test, percent)
#partition_directory(n_directory, n_test, percent)
#partition_directory(s_directory, s_test, percent)
#partition_directory(o_directory, o_test, percent)

D://Datasets//TRAINING//Happy_test//Happy1291.wav
D://Datasets//TRAINING//Happy_test//Happy691.wav
D://Datasets//TRAINING//Happy_test//Happy1303.wav
D://Datasets//TRAINING//Happy_test//Happy1491.wav
D://Datasets//TRAINING//Happy_test//Happy245.wav
D://Datasets//TRAINING//Happy_test//Happy1506.wav
D://Datasets//TRAINING//Happy_test//Happy1779.wav
D://Datasets//TRAINING//Happy_test//Happy835.wav
D://Datasets//TRAINING//Happy_test//Happy93.wav
D://Datasets//TRAINING//Happy_test//Happy569.wav
D://Datasets//TRAINING//Happy_test//Happy994.wav
D://Datasets//TRAINING//Happy_test//Happy154.wav
D://Datasets//TRAINING//Happy_test//Happy290.wav
D://Datasets//TRAINING//Happy_test//Happy1137.wav
D://Datasets//TRAINING//Happy_test//Happy1231.wav
D://Datasets//TRAINING//Happy_test//Happy228.wav
D://Datasets//TRAINING//Happy_test//Happy67.wav
D://Datasets//TRAINING//Happy_test//Happy1102.wav
D://Datasets//TRAINING//Happy_test//Happy695.wav
D://Datasets//TRAINING//Happy_test//Happy1.wav
D://Datasets//TR

D://Datasets//TRAINING//Happy_test//Happy253.wav
D://Datasets//TRAINING//Happy_test//Happy1603.wav
D://Datasets//TRAINING//Happy_test//Happy1185.wav
D://Datasets//TRAINING//Happy_test//Happy929.wav
D://Datasets//TRAINING//Happy_test//Happy135.wav
D://Datasets//TRAINING//Happy_test//Happy1672.wav
D://Datasets//TRAINING//Happy_test//Happy1145.wav
D://Datasets//TRAINING//Happy_test//Happy192.wav
D://Datasets//TRAINING//Happy_test//Happy1056.wav
D://Datasets//TRAINING//Happy_test//Happy581.wav
D://Datasets//TRAINING//Happy_test//Happy729.wav
D://Datasets//TRAINING//Happy_test//Happy1284.wav
D://Datasets//TRAINING//Happy_test//Happy1165.wav
D://Datasets//TRAINING//Happy_test//Happy4.wav
D://Datasets//TRAINING//Happy_test//Happy1788.wav
D://Datasets//TRAINING//Happy_test//Happy1071.wav
D://Datasets//TRAINING//Happy_test//Happy26.wav
D://Datasets//TRAINING//Happy_test//Happy1638.wav
D://Datasets//TRAINING//Happy_test//Happy872.wav
D://Datasets//TRAINING//Happy_test//Happy272.wav
D://Datasets/

In [49]:
def pad_silence_per_file(emotionfile, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    file_duration = emotionsound.duration_seconds * 1000
    silence = AudioSegment.silent(duration=5000)
    threshold = 5000 - file_duration
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        emotionsound = emotionsound[0:5000]
        overlay_start = 0
    
    newSound = silence.overlay(emotionsound, position=overlay_start)
    
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

In [50]:
def pad_silence_per_folder(directory, extension):
    for file in os.listdir(directory):
        if file.endswith(extension) and file[0] != '.':
            newSoundFile = directory + file
            pad_silence_per_file(newSoundFile, newSoundFile)
            print(newSoundFile)

In [51]:
'''
for path in emotion_train_folders:
    pad_silence_per_folder(path, '.wav')
    
for path in emotion_test_folders:
    pad_silence_per_folder(path, '.wav')
'''
#pad_silence_per_folder(h_directory, '.wav')
#pad_silence_per_folder(h_test, '.wav')

D://Datasets//TRAINING//Happy//Happy374.wav
D://Datasets//TRAINING//Happy//Happy375.wav
D://Datasets//TRAINING//Happy//Happy376.wav
D://Datasets//TRAINING//Happy//Happy377.wav
D://Datasets//TRAINING//Happy//Happy378.wav
D://Datasets//TRAINING//Happy//Happy379.wav
D://Datasets//TRAINING//Happy//Happy380.wav
D://Datasets//TRAINING//Happy//Happy381.wav
D://Datasets//TRAINING//Happy//Happy382.wav
D://Datasets//TRAINING//Happy//Happy383.wav
D://Datasets//TRAINING//Happy//Happy384.wav
D://Datasets//TRAINING//Happy//Happy385.wav
D://Datasets//TRAINING//Happy//Happy386.wav
D://Datasets//TRAINING//Happy//Happy388.wav
D://Datasets//TRAINING//Happy//Happy390.wav
D://Datasets//TRAINING//Happy//Happy391.wav
D://Datasets//TRAINING//Happy//Happy392.wav
D://Datasets//TRAINING//Happy//Happy393.wav
D://Datasets//TRAINING//Happy//Happy394.wav
D://Datasets//TRAINING//Happy//Happy395.wav
D://Datasets//TRAINING//Happy//Happy396.wav
D://Datasets//TRAINING//Happy//Happy397.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy603.wav
D://Datasets//TRAINING//Happy//Happy171.wav
D://Datasets//TRAINING//Happy//Happy605.wav
D://Datasets//TRAINING//Happy//Happy606.wav
D://Datasets//TRAINING//Happy//Happy608.wav
D://Datasets//TRAINING//Happy//Happy609.wav
D://Datasets//TRAINING//Happy//Happy610.wav
D://Datasets//TRAINING//Happy//Happy177.wav
D://Datasets//TRAINING//Happy//Happy612.wav
D://Datasets//TRAINING//Happy//Happy613.wav
D://Datasets//TRAINING//Happy//Happy614.wav
D://Datasets//TRAINING//Happy//Happy615.wav
D://Datasets//TRAINING//Happy//Happy617.wav
D://Datasets//TRAINING//Happy//Happy619.wav
D://Datasets//TRAINING//Happy//Happy620.wav
D://Datasets//TRAINING//Happy//Happy621.wav
D://Datasets//TRAINING//Happy//Happy622.wav
D://Datasets//TRAINING//Happy//Happy623.wav
D://Datasets//TRAINING//Happy//Happy624.wav
D://Datasets//TRAINING//Happy//Happy625.wav
D://Datasets//TRAINING//Happy//Happy626.wav
D://Datasets//TRAINING//Happy//Happy627.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy829.wav
D://Datasets//TRAINING//Happy//Happy830.wav
D://Datasets//TRAINING//Happy//Happy831.wav
D://Datasets//TRAINING//Happy//Happy832.wav
D://Datasets//TRAINING//Happy//Happy833.wav
D://Datasets//TRAINING//Happy//Happy837.wav
D://Datasets//TRAINING//Happy//Happy838.wav
D://Datasets//TRAINING//Happy//Happy839.wav
D://Datasets//TRAINING//Happy//Happy842.wav
D://Datasets//TRAINING//Happy//Happy843.wav
D://Datasets//TRAINING//Happy//Happy844.wav
D://Datasets//TRAINING//Happy//Happy845.wav
D://Datasets//TRAINING//Happy//Happy846.wav
D://Datasets//TRAINING//Happy//Happy847.wav
D://Datasets//TRAINING//Happy//Happy848.wav
D://Datasets//TRAINING//Happy//Happy849.wav
D://Datasets//TRAINING//Happy//Happy850.wav
D://Datasets//TRAINING//Happy//Happy851.wav
D://Datasets//TRAINING//Happy//Happy852.wav
D://Datasets//TRAINING//Happy//Happy853.wav
D://Datasets//TRAINING//Happy//Happy854.wav
D://Datasets//TRAINING//Happy//Happy855.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy1049.wav
D://Datasets//TRAINING//Happy//Happy1052.wav
D://Datasets//TRAINING//Happy//Happy1053.wav
D://Datasets//TRAINING//Happy//Happy1054.wav
D://Datasets//TRAINING//Happy//Happy1055.wav
D://Datasets//TRAINING//Happy//Happy1057.wav
D://Datasets//TRAINING//Happy//Happy1058.wav
D://Datasets//TRAINING//Happy//Happy1059.wav
D://Datasets//TRAINING//Happy//Happy1060.wav
D://Datasets//TRAINING//Happy//Happy1061.wav
D://Datasets//TRAINING//Happy//Happy1062.wav
D://Datasets//TRAINING//Happy//Happy1063.wav
D://Datasets//TRAINING//Happy//Happy1064.wav
D://Datasets//TRAINING//Happy//Happy1065.wav
D://Datasets//TRAINING//Happy//Happy1066.wav
D://Datasets//TRAINING//Happy//Happy1067.wav
D://Datasets//TRAINING//Happy//Happy1069.wav
D://Datasets//TRAINING//Happy//Happy1070.wav
D://Datasets//TRAINING//Happy//Happy1072.wav
D://Datasets//TRAINING//Happy//Happy1073.wav
D://Datasets//TRAINING//Happy//Happy1074.wav
D://Datasets//TRAINING//Happy//Happy1076.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy1279.wav
D://Datasets//TRAINING//Happy//Happy1280.wav
D://Datasets//TRAINING//Happy//Happy1281.wav
D://Datasets//TRAINING//Happy//Happy1282.wav
D://Datasets//TRAINING//Happy//Happy1285.wav
D://Datasets//TRAINING//Happy//Happy700.wav
D://Datasets//TRAINING//Happy//Happy1286.wav
D://Datasets//TRAINING//Happy//Happy1287.wav
D://Datasets//TRAINING//Happy//Happy1288.wav
D://Datasets//TRAINING//Happy//Happy1289.wav
D://Datasets//TRAINING//Happy//Happy1292.wav
D://Datasets//TRAINING//Happy//Happy1293.wav
D://Datasets//TRAINING//Happy//Happy1294.wav
D://Datasets//TRAINING//Happy//Happy1295.wav
D://Datasets//TRAINING//Happy//Happy1296.wav
D://Datasets//TRAINING//Happy//Happy1297.wav
D://Datasets//TRAINING//Happy//Happy1298.wav
D://Datasets//TRAINING//Happy//Happy711.wav
D://Datasets//TRAINING//Happy//Happy1299.wav
D://Datasets//TRAINING//Happy//Happy1300.wav
D://Datasets//TRAINING//Happy//Happy1301.wav
D://Datasets//TRAINING//Happy//Happy1302.wav
D://Datasets

D://Datasets//TRAINING//Happy//Happy1504.wav
D://Datasets//TRAINING//Happy//Happy1505.wav
D://Datasets//TRAINING//Happy//Happy1507.wav
D://Datasets//TRAINING//Happy//Happy1508.wav
D://Datasets//TRAINING//Happy//Happy871.wav
D://Datasets//TRAINING//Happy//Happy1509.wav
D://Datasets//TRAINING//Happy//Happy1510.wav
D://Datasets//TRAINING//Happy//Happy1512.wav
D://Datasets//TRAINING//Happy//Happy1513.wav
D://Datasets//TRAINING//Happy//Happy1514.wav
D://Datasets//TRAINING//Happy//Happy1516.wav
D://Datasets//TRAINING//Happy//Happy1517.wav
D://Datasets//TRAINING//Happy//Happy1519.wav
D://Datasets//TRAINING//Happy//Happy1520.wav
D://Datasets//TRAINING//Happy//Happy1521.wav
D://Datasets//TRAINING//Happy//Happy1522.wav
D://Datasets//TRAINING//Happy//Happy1523.wav
D://Datasets//TRAINING//Happy//Happy1526.wav
D://Datasets//TRAINING//Happy//Happy1527.wav
D://Datasets//TRAINING//Happy//Happy1528.wav
D://Datasets//TRAINING//Happy//Happy1529.wav
D://Datasets//TRAINING//Happy//Happy1531.wav
D://Dataset

D://Datasets//TRAINING//Happy//Happy1739.wav
D://Datasets//TRAINING//Happy//Happy1740.wav
D://Datasets//TRAINING//Happy//Happy1741.wav
D://Datasets//TRAINING//Happy//Happy1743.wav
D://Datasets//TRAINING//Happy//Happy1744.wav
D://Datasets//TRAINING//Happy//Happy1745.wav
D://Datasets//TRAINING//Happy//Happy1747.wav
D://Datasets//TRAINING//Happy//Happy1748.wav
D://Datasets//TRAINING//Happy//Happy1749.wav
D://Datasets//TRAINING//Happy//Happy1750.wav
D://Datasets//TRAINING//Happy//Happy1751.wav
D://Datasets//TRAINING//Happy//Happy1752.wav
D://Datasets//TRAINING//Happy//Happy1753.wav
D://Datasets//TRAINING//Happy//Happy1754.wav
D://Datasets//TRAINING//Happy//Happy1755.wav
D://Datasets//TRAINING//Happy//Happy1756.wav
D://Datasets//TRAINING//Happy//Happy1757.wav
D://Datasets//TRAINING//Happy//Happy1758.wav
D://Datasets//TRAINING//Happy//Happy1759.wav
D://Datasets//TRAINING//Happy//Happy1761.wav
D://Datasets//TRAINING//Happy//Happy1762.wav
D://Datasets//TRAINING//Happy//Happy1764.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy172.wav
D://Datasets//TRAINING//Happy//Happy174.wav
D://Datasets//TRAINING//Happy//Happy175.wav
D://Datasets//TRAINING//Happy//Happy176.wav
D://Datasets//TRAINING//Happy//Happy178.wav
D://Datasets//TRAINING//Happy//Happy180.wav
D://Datasets//TRAINING//Happy//Happy181.wav
D://Datasets//TRAINING//Happy//Happy182.wav
D://Datasets//TRAINING//Happy//Happy183.wav
D://Datasets//TRAINING//Happy//Happy184.wav
D://Datasets//TRAINING//Happy//Happy186.wav
D://Datasets//TRAINING//Happy//Happy187.wav
D://Datasets//TRAINING//Happy//Happy188.wav
D://Datasets//TRAINING//Happy//Happy189.wav
D://Datasets//TRAINING//Happy//Happy190.wav
D://Datasets//TRAINING//Happy//Happy191.wav
D://Datasets//TRAINING//Happy//Happy194.wav
D://Datasets//TRAINING//Happy//Happy196.wav
D://Datasets//TRAINING//Happy//Happy197.wav
D://Datasets//TRAINING//Happy//Happy198.wav
D://Datasets//TRAINING//Happy//Happy199.wav
D://Datasets//TRAINING//Happy//Happy200.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy_test//Happy567.wav
D://Datasets//TRAINING//Happy_test//Happy1437.wav
D://Datasets//TRAINING//Happy_test//Happy601.wav
D://Datasets//TRAINING//Happy_test//Happy1088.wav
D://Datasets//TRAINING//Happy_test//Happy1227.wav
D://Datasets//TRAINING//Happy_test//Happy673.wav
D://Datasets//TRAINING//Happy_test//Happy952.wav
D://Datasets//TRAINING//Happy_test//Happy657.wav
D://Datasets//TRAINING//Happy_test//Happy236.wav
D://Datasets//TRAINING//Happy_test//Happy489.wav
D://Datasets//TRAINING//Happy_test//Happy561.wav
D://Datasets//TRAINING//Happy_test//Happy1530.wav
D://Datasets//TRAINING//Happy_test//Happy527.wav
D://Datasets//TRAINING//Happy_test//Happy1683.wav
D://Datasets//TRAINING//Happy_test//Happy1200.wav
D://Datasets//TRAINING//Happy_test//Happy1566.wav
D://Datasets//TRAINING//Happy_test//Happy1079.wav
D://Datasets//TRAINING//Happy_test//Happy684.wav
D://Datasets//TRAINING//Happy_test//Happy1366.wav
D://Datasets//TRAINING//Happy_test//Happy575.wav
D://Dataset

D://Datasets//TRAINING//Happy_test//Happy1190.wav
D://Datasets//TRAINING//Happy_test//Happy1511.wav
D://Datasets//TRAINING//Happy_test//Happy1365.wav
D://Datasets//TRAINING//Happy_test//Happy1669.wav
D://Datasets//TRAINING//Happy_test//Happy225.wav
D://Datasets//TRAINING//Happy_test//Happy1780.wav
D://Datasets//TRAINING//Happy_test//Happy814.wav
D://Datasets//TRAINING//Happy_test//Happy1068.wav
D://Datasets//TRAINING//Happy_test//Happy265.wav
D://Datasets//TRAINING//Happy_test//Happy1252.wav
D://Datasets//TRAINING//Happy_test//Happy566.wav
D://Datasets//TRAINING//Happy_test//Happy1579.wav
D://Datasets//TRAINING//Happy_test//Happy1671.wav
D://Datasets//TRAINING//Happy_test//Happy1427.wav
D://Datasets//TRAINING//Happy_test//Happy1151.wav
D://Datasets//TRAINING//Happy_test//Happy1304.wav
D://Datasets//TRAINING//Happy_test//Happy1386.wav
D://Datasets//TRAINING//Happy_test//Happy462.wav
D://Datasets//TRAINING//Happy_test//Happy962.wav
D://Datasets//TRAINING//Happy_test//Happy1570.wav
D://Da

In [73]:
def change_amplitude(emotionfile, d1, newSoundFile, d2):
    
    if d1 <= d2:
        sound = AudioSegment.from_file(emotionfile) - np.random.randint(0, (6 * d2/d1 - 1))
        sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    else:
        print('Invalid distance parameters. d1 should be <= d2.')

In [74]:
def change_amplitude_range(emotionfile, newSoundFile, threshold):
    #amount = np.random.randint(0, threshold)
    amount = random.uniform(0, threshold)
    #print('Deamplify ' + str(emotionfile) + ' by ' + str(amount) + ' db.')
    sound = AudioSegment.from_file(emotionfile) - amount
    sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    return amount

In [77]:
'''
def deamplify_per_folder(directory):
    for file in os.listdir(directory):
        if file.endswith('.wav'):
            soundFile = directory + file
            newSoundFile = directory + 'deamplified_' + file
            change_amplitude_range(soundFile, newSoundFile, 12)
'''

"\ndef deamplify_per_folder(directory):\n    for file in os.listdir(directory):\n        if file.endswith('.wav'):\n            soundFile = directory + file\n            newSoundFile = directory + 'deamplified_' + file\n            change_amplitude_range(soundFile, newSoundFile, 12)\n"

In [78]:
'''
for directory in emotion_train_folders:
    deamplify_per_folder(directory)
    
for directory in emotion_test_folders:
    deamplify_per_folder(directory)
'''

'\nfor directory in emotion_train_folders:\n    deamplify_per_folder(directory)\n    \nfor directory in emotion_test_folders:\n    deamplify_per_folder(directory)\n'

In [79]:
def add_noise_per_file(emotionfile, bgnoise, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    emotion_duration = emotionsound.duration_seconds * 1000
    noise = AudioSegment.from_wav(bgnoise)
    noise_duration = noise.duration_seconds * 1000
    
    threshold = noise_duration - emotion_duration
    
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        overlay_start = 0
        
    targeted_chunk = noise[overlay_start:overlay_start + emotion_duration]
    newSound = emotionsound.overlay(targeted_chunk, position=0)
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

In [80]:
'''
def add_noise_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 3):
                soundFile = directory + file
                newSoundFile = directory + str(i) + '_' + file
                random_noise = noise_directory + random.choice(os.listdir(noise_directory))

                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)
'''

"\ndef add_noise_per_folder(directory, extension, noise_directory):\n    for file in os.listdir(directory):\n        if file.endswith(extension) and not file[1] == '_':\n            for i in range(0, 3):\n                soundFile = directory + file\n                newSoundFile = directory + str(i) + '_' + file\n                random_noise = noise_directory + random.choice(os.listdir(noise_directory))\n\n                add_noise_per_file(soundFile, random_noise, newSoundFile)\n                print(newSoundFile)\n"

In [81]:
def add_noise_and_deamplify_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 2):
                soundFile = directory + file
                amount = change_amplitude_range(soundFile, soundFile, 12)
                noise = random.choice(os.listdir(noise_directory))
                random_noise = noise_directory + noise
                newSoundFile = directory + 'deamp_' + str(amount) + '_noise_' + noise[:len(noise)-5] + '_' + file
                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)

            

In [82]:
#add_noise_and_deamplify_per_folder('C://Users//yg9ca//Documents//Datasets//TEST//EMOTION//', 'wav', background_noise)

In [83]:
add_noise_and_deamplify_per_folder(h_directory, 'wav', background_noise)
add_noise_and_deamplify_per_folder(a_directory, 'wav', background_noise)
add_noise_and_deamplify_per_folder(n_directory, 'wav', background_noise)
add_noise_and_deamplify_per_folder(s_directory, 'wav', background_noise)
add_noise_and_deamplify_per_folder(o_directory, 'wav', background_noise)

D://Datasets//TRAINING//Happy//deamp_10.604645203909602_noise_residential_area_3_Happy374.wav
D://Datasets//TRAINING//Happy//deamp_0.22589981673210424_noise_car_17_Happy374.wav
D://Datasets//TRAINING//Happy//deamp_1.8917498686684522_noise_grocery_store_75_Happy375.wav
D://Datasets//TRAINING//Happy//deamp_10.516106032872248_noise_car_80_Happy375.wav
D://Datasets//TRAINING//Happy//deamp_8.493747101296304_noise_library_58_Happy376.wav
D://Datasets//TRAINING//Happy//deamp_4.183201410177844_noise_park_52_Happy376.wav
D://Datasets//TRAINING//Happy//deamp_3.518053344460589_noise_park_9_Happy377.wav
D://Datasets//TRAINING//Happy//deamp_11.647594623427318_noise_office_88_Happy377.wav
D://Datasets//TRAINING//Happy//deamp_9.585284550338535_noise_park_9_Happy378.wav
D://Datasets//TRAINING//Happy//deamp_3.64967049215222_noise_a130_210_24_Happy378.wav
D://Datasets//TRAINING//Happy//deamp_9.369154435534004_noise_metro_station_71_Happy379.wav
D://Datasets//TRAINING//Happy//deamp_8.4794449868215_noise_

D://Datasets//TRAINING//Happy//deamp_8.947316481263176_noise_bus_84_Happy428.wav
D://Datasets//TRAINING//Happy//deamp_1.221555794922058_noise_library_62_Happy428.wav
D://Datasets//TRAINING//Happy//deamp_0.9072737682129794_noise_library_57_Happy430.wav
D://Datasets//TRAINING//Happy//deamp_9.917260606836887_noise_library_59_Happy430.wav
D://Datasets//TRAINING//Happy//deamp_4.659380709790968_noise_tram_72_Happy431.wav
D://Datasets//TRAINING//Happy//deamp_11.614673738073137_noise_beach_51_Happy431.wav
D://Datasets//TRAINING//Happy//deamp_10.512798157950678_noise_residential_area_45_Happy432.wav
D://Datasets//TRAINING//Happy//deamp_3.028281991823233_noise_metro_station_71_Happy432.wav
D://Datasets//TRAINING//Happy//deamp_3.4960262885081437_noise_office_64_Happy433.wav
D://Datasets//TRAINING//Happy//deamp_11.006485646099147_noise_beach_1_Happy433.wav
D://Datasets//TRAINING//Happy//deamp_11.483662984975474_noise_car_23_Happy434.wav
D://Datasets//TRAINING//Happy//deamp_4.883635878223944_noise_

D://Datasets//TRAINING//Happy//deamp_8.281869397989732_noise_tram_21_Happy484.wav
D://Datasets//TRAINING//Happy//deamp_4.730689533321317_noise_library_59_Happy485.wav
D://Datasets//TRAINING//Happy//deamp_6.180137963650322_noise_tram_38_Happy485.wav
D://Datasets//TRAINING//Happy//deamp_11.02183776220016_noise_a131_0_3_Happy486.wav
D://Datasets//TRAINING//Happy//deamp_10.859257715112744_noise_home_9_Happy486.wav
D://Datasets//TRAINING//Happy//deamp_7.653631532972138_noise_bus_14_Happy487.wav
D://Datasets//TRAINING//Happy//deamp_10.020474359354676_noise_home_11_Happy487.wav
D://Datasets//TRAINING//Happy//deamp_9.512330101887233_noise_metro_station_70_Happy488.wav
D://Datasets//TRAINING//Happy//deamp_4.106149655657319_noise_city_center_78_Happy488.wav
D://Datasets//TRAINING//Happy//deamp_7.393841428089656_noise_library_61_Happy490.wav
D://Datasets//TRAINING//Happy//deamp_8.581076783014595_noise_office_88_Happy490.wav
D://Datasets//TRAINING//Happy//deamp_3.268318355321839_noise_residential_

D://Datasets//TRAINING//Happy//deamp_3.947945456594478_noise_park_5_Happy544.wav
D://Datasets//TRAINING//Happy//deamp_9.697465421102688_noise_office_88_Happy545.wav
D://Datasets//TRAINING//Happy//deamp_1.5964631526195299_noise_b056_60_9_Happy545.wav
D://Datasets//TRAINING//Happy//deamp_5.11961310159691_noise_bus_27_Happy546.wav
D://Datasets//TRAINING//Happy//deamp_3.302105634386253_noise_beach_51_Happy546.wav
D://Datasets//TRAINING//Happy//deamp_9.333893624959504_noise_library_60_Happy547.wav
D://Datasets//TRAINING//Happy//deamp_1.0344406014819718_noise_city_center_79_Happy547.wav
D://Datasets//TRAINING//Happy//deamp_7.406623569700446_noise_metro_station_42_Happy548.wav
D://Datasets//TRAINING//Happy//deamp_2.4091309511435073_noise_park_48_Happy548.wav
D://Datasets//TRAINING//Happy//deamp_4.747613509927408_noise_metro_station_70_Happy549.wav
D://Datasets//TRAINING//Happy//deamp_7.807813570543313_noise_bus_84_Happy549.wav
D://Datasets//TRAINING//Happy//deamp_5.3893447066249145_noise_park

D://Datasets//TRAINING//Happy//deamp_3.568816255539962_noise_tram_38_Happy606.wav
D://Datasets//TRAINING//Happy//deamp_6.1855187121911985_noise_city_center_1_Happy608.wav
D://Datasets//TRAINING//Happy//deamp_11.716794301786909_noise_bus_84_Happy608.wav
D://Datasets//TRAINING//Happy//deamp_10.114298237719531_noise_residential_area__Happy609.wav
D://Datasets//TRAINING//Happy//deamp_1.7986656776872199_noise_beach_8_Happy609.wav
D://Datasets//TRAINING//Happy//deamp_0.07402245523427986_noise_train_19_Happy610.wav
D://Datasets//TRAINING//Happy//deamp_11.577251658060078_noise_tram_72_Happy610.wav
D://Datasets//TRAINING//Happy//deamp_9.213898887292883_noise_park_52_Happy177.wav
D://Datasets//TRAINING//Happy//deamp_4.502952698678512_noise_forest_path_25_Happy177.wav
D://Datasets//TRAINING//Happy//deamp_1.054343331575632_noise_beach_8_Happy612.wav
D://Datasets//TRAINING//Happy//deamp_11.845312043965798_noise_beach_47_Happy612.wav
D://Datasets//TRAINING//Happy//deamp_9.964083673711553_noise_city_

...

D://Datasets//TRAINING//Other//deamp_1.6555411861278024_noise_forest_path_25_Other2370.wav
D://Datasets//TRAINING//Other//deamp_5.542113239825422_noise_office_88_Other2371.wav
D://Datasets//TRAINING//Other//deamp_4.312407240907875_noise_office_26_Other2371.wav
D://Datasets//TRAINING//Other//deamp_3.1960823418244693_noise_metro_station_42_Other2372.wav
D://Datasets//TRAINING//Other//deamp_5.636614617696861_noise_residential_area_46_Other2372.wav
D://Datasets//TRAINING//Other//deamp_8.075766398806914_noise_forest_path_17_Other2374.wav
D://Datasets//TRAINING//Other//deamp_6.0092930567894625_noise_tram_37_Other2374.wav
D://Datasets//TRAINING//Other//deamp_4.20301970526204_noise_metro_station_21_Other2375.wav
D://Datasets//TRAINING//Other//deamp_11.025507445490266_noise_car_81_Other2375.wav
D://Datasets//TRAINING//Other//deamp_7.682397971225857_noise_train_19_Other2376.wav
D://Datasets//TRAINING//Other//deamp_7.781891856180944_noise_park_6_Other2376.wav
D://Datasets//TRAINING//Other//deamp_

D://Datasets//TRAINING//Other//deamp_11.782484190863812_noise_city_center_81_Other2427.wav
D://Datasets//TRAINING//Other//deamp_6.062829323969182_noise_office_26_Other2427.wav
D://Datasets//TRAINING//Other//deamp_3.8594971676301832_noise_tram_39_Other2428.wav
D://Datasets//TRAINING//Other//deamp_8.158506707625005_noise_bus_86_Other2428.wav
D://Datasets//TRAINING//Other//deamp_9.961343236540626_noise_office_30_Other2429.wav
D://Datasets//TRAINING//Other//deamp_4.787320926868703_noise_office_32_Other2429.wav
D://Datasets//TRAINING//Other//deamp_5.164147195214516_noise_office_31_Other2430.wav
D://Datasets//TRAINING//Other//deamp_2.913887336172053_noise_beach_51_Other2430.wav
D://Datasets//TRAINING//Other//deamp_6.338463362579919_noise_bus_13_Other2432.wav
D://Datasets//TRAINING//Other//deamp_5.115726208243737_noise_park_52_Other2432.wav
D://Datasets//TRAINING//Other//deamp_7.885934111045467_noise_tram_38_Other2433.wav
D://Datasets//TRAINING//Other//deamp_1.8028032272990693_noise_tram_21_O

D://Datasets//TRAINING//Other//deamp_8.26833069126435_noise_grocery_store_28_Other2482.wav
D://Datasets//TRAINING//Other//deamp_2.318177848710486_noise_office_64_Other2484.wav
D://Datasets//TRAINING//Other//deamp_8.612930683568901_noise_library_59_Other2484.wav
D://Datasets//TRAINING//Other//deamp_9.615667179712085_noise_car_34_Other2485.wav
D://Datasets//TRAINING//Other//deamp_8.32466937382214_noise_city_center_1_Other2485.wav
D://Datasets//TRAINING//Other//deamp_3.861976720679696_noise_park_5_Other2487.wav
D://Datasets//TRAINING//Other//deamp_5.907679353419617_noise_tram_39_Other2487.wav
D://Datasets//TRAINING//Other//deamp_5.901477228053617_noise_city_center_78_Other2488.wav
D://Datasets//TRAINING//Other//deamp_10.21709727503374_noise_a082_30_6_Other2488.wav
D://Datasets//TRAINING//Other//deamp_0.08749158671521284_noise_tram_40_Other2489.wav
D://Datasets//TRAINING//Other//deamp_2.8970698169907654_noise_bus_86_Other2489.wav
D://Datasets//TRAINING//Other//deamp_11.944177739260326_nois

D://Datasets//TRAINING//Other//deamp_3.1468251032716_noise_car_34_Other2535.wav
D://Datasets//TRAINING//Other//deamp_3.4179594225846555_noise_bus_86_Other2535.wav
D://Datasets//TRAINING//Other//deamp_0.08069863394223553_noise_beach_4_Other2537.wav
D://Datasets//TRAINING//Other//deamp_2.39086628571237_noise_library_57_Other2537.wav
D://Datasets//TRAINING//Other//deamp_11.793898285303916_noise_office_88_Other2540.wav
D://Datasets//TRAINING//Other//deamp_10.189494965576916_noise_grocery_store_83_Other2540.wav


CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

b'ffmpeg version 2.7 Copyright (c) 2000-2015 the FFmpeg developers\r\n  built with gcc 4.9.2 (GCC)\r\n  configuration: --enable-gpl --enable-version3 --disable-w32threads --enable-avisynth --enable-bzlib --enable-fontconfig --enable-frei0r --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libdcadec --enable-libfreetype --enable-libgme --enable-libgsm --enable-libilbc --enable-libmodplug --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-librtmp --enable-libschroedinger --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvo-aacenc --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-lzma --enable

In [84]:
add_noise_and_deamplify_per_folder(h_test, '.wav', background_noise_test)

D://Datasets//TRAINING//Happy_test//deamp_1.687049524672401_noise_bus_18_Happy1291.wav
D://Datasets//TRAINING//Happy_test//deamp_3.7073735120723135_noise_tram_9_Happy1291.wav
D://Datasets//TRAINING//Happy_test//deamp_3.7159496048348775_noise_residential_area__Happy691.wav
D://Datasets//TRAINING//Happy_test//deamp_11.842492822256613_noise_car_5_Happy691.wav
D://Datasets//TRAINING//Happy_test//deamp_4.746528470035297_noise_home_2_Happy1303.wav
D://Datasets//TRAINING//Happy_test//deamp_7.289820329048595_noise_car_8_Happy1303.wav
D://Datasets//TRAINING//Happy_test//deamp_0.6261054923799052_noise_train_5_Happy1491.wav
D://Datasets//TRAINING//Happy_test//deamp_5.263366171473557_noise_tram_16_Happy1491.wav
D://Datasets//TRAINING//Happy_test//deamp_0.47040854831509593_noise_residential_area__Happy245.wav
D://Datasets//TRAINING//Happy_test//deamp_6.075674581031047_noise_train_8_Happy245.wav
D://Datasets//TRAINING//Happy_test//deamp_7.972864889449847_noise_home_3_Happy1506.wav
D://Datasets//TRAI

D://Datasets//TRAINING//Happy_test//deamp_10.953757084004838_noise_car_18_Happy1079.wav
D://Datasets//TRAINING//Happy_test//deamp_4.749109528262013_noise_grocery_store_18_Happy684.wav
D://Datasets//TRAINING//Happy_test//deamp_1.5731525932235955_noise_bus_19_Happy684.wav
D://Datasets//TRAINING//Happy_test//deamp_3.421325163167777_noise_train_4_Happy1366.wav
D://Datasets//TRAINING//Happy_test//deamp_1.7142876517818784_noise_city_center_17_Happy1366.wav
D://Datasets//TRAINING//Happy_test//deamp_3.6692876547046276_noise_library_12_Happy575.wav
D://Datasets//TRAINING//Happy_test//deamp_0.3908966014739277_noise_train_14_Happy575.wav
D://Datasets//TRAINING//Happy_test//deamp_2.5043624519183334_noise_beach_11_Happy1256.wav
D://Datasets//TRAINING//Happy_test//deamp_11.38668019188668_noise_train_15_Happy1256.wav
D://Datasets//TRAINING//Happy_test//deamp_9.378584029110364_noise_home_2_Happy344.wav
D://Datasets//TRAINING//Happy_test//deamp_1.4891777052556945_noise_bus_19_Happy344.wav
D://Datasets/

D://Datasets//TRAINING//Happy_test//deamp_9.521834850371453_noise_train_8_Happy1383.wav
D://Datasets//TRAINING//Happy_test//deamp_5.990426260868174_noise_beach_1_Happy1383.wav
D://Datasets//TRAINING//Happy_test//deamp_1.854853066660099_noise_tram_16_Happy495.wav
D://Datasets//TRAINING//Happy_test//deamp_9.521786644635995_noise_beach_11_Happy495.wav
D://Datasets//TRAINING//Happy_test//deamp_5.755170740047555_noise_grocery_store_17_Happy983.wav
D://Datasets//TRAINING//Happy_test//deamp_11.771730523635835_noise_city_center_18_Happy983.wav
D://Datasets//TRAINING//Happy_test//deamp_2.39051791166489_noise_train_15_Happy18.wav
D://Datasets//TRAINING//Happy_test//deamp_6.0931848532382675_noise_city_center_17_Happy18.wav
D://Datasets//TRAINING//Happy_test//deamp_11.561369606046581_noise_residential_area_9_Happy1679.wav
D://Datasets//TRAINING//Happy_test//deamp_10.08849351588194_noise_forest_path_3_Happy1679.wav
D://Datasets//TRAINING//Happy_test//deamp_8.232000029704405_noise_metro_station_9_Ha

D://Datasets//TRAINING//Happy_test//deamp_0.5694161803034716_noise_beach_1_Happy963.wav
D://Datasets//TRAINING//Happy_test//deamp_0.3114786131672722_noise_office_19_Happy655.wav
D://Datasets//TRAINING//Happy_test//deamp_4.966270057278985_noise_tram_16_Happy655.wav
D://Datasets//TRAINING//Happy_test//deamp_9.107498464636164_noise_car_13_Happy255.wav
D://Datasets//TRAINING//Happy_test//deamp_3.205420179277822_noise_residential_area__Happy255.wav
D://Datasets//TRAINING//Happy_test//deamp_6.036960387223228_noise_a107_30_6_Happy1305.wav
D://Datasets//TRAINING//Happy_test//deamp_1.8881431476574981_noise_bus_18_Happy1305.wav
D://Datasets//TRAINING//Happy_test//deamp_9.947663476111764_noise_park_2_Happy1210.wav
D://Datasets//TRAINING//Happy_test//deamp_2.296502465254107_noise_bus_4_Happy1210.wav
D://Datasets//TRAINING//Happy_test//deamp_2.7980560881863563_noise_grocery_store_17_Happy716.wav
D://Datasets//TRAINING//Happy_test//deamp_10.211111865408952_noise_a107_120_15_Happy716.wav
D://Datasets

D://Datasets//TRAINING//Happy_test//deamp_10.351339352551785_noise_beach_11_Happy26.wav
D://Datasets//TRAINING//Happy_test//deamp_2.725617801016406_noise_bus_8_Happy26.wav
D://Datasets//TRAINING//Happy_test//deamp_1.1393945245844321_noise_park_2_Happy1638.wav
D://Datasets//TRAINING//Happy_test//deamp_4.731207177476585_noise_train_14_Happy1638.wav
D://Datasets//TRAINING//Happy_test//deamp_10.254816162069934_noise_grocery_store_6_Happy872.wav
D://Datasets//TRAINING//Happy_test//deamp_0.024618603478490986_noise_bus_19_Happy872.wav
D://Datasets//TRAINING//Happy_test//deamp_0.5986898486891761_noise_city_center_7_Happy272.wav
D://Datasets//TRAINING//Happy_test//deamp_10.826446505569585_noise_home_3_Happy272.wav
D://Datasets//TRAINING//Happy_test//deamp_2.016355053836368_noise_grocery_store_17_Happy429.wav
D://Datasets//TRAINING//Happy_test//deamp_10.870402998284169_noise_a107_120_15_Happy429.wav
D://Datasets//TRAINING//Happy_test//deamp_7.858224742575009_noise_metro_station_5_Happy1717.wav
D

D://Datasets//TRAINING//Happy_test//deamp_11.294668346245814_noise_library_13_Happy1490.wav
D://Datasets//TRAINING//Happy_test//deamp_8.624105620963507_noise_home_3_Happy913.wav
D://Datasets//TRAINING//Happy_test//deamp_4.0877029897110235_noise_office_19_Happy913.wav
D://Datasets//TRAINING//Happy_test//deamp_7.026550963525256_noise_grocery_store_17_Happy1012.wav
D://Datasets//TRAINING//Happy_test//deamp_3.149629094103959_noise_forest_path_3_Happy1012.wav
D://Datasets//TRAINING//Happy_test//deamp_0.3000281688974349_noise_metro_station_5_Happy1746.wav
D://Datasets//TRAINING//Happy_test//deamp_8.527216178681803_noise_beach_11_Happy1746.wav
D://Datasets//TRAINING//Happy_test//deamp_0.27364995923803503_noise_car_18_Happy675.wav
D://Datasets//TRAINING//Happy_test//deamp_8.515732632997999_noise_car_18_Happy675.wav
D://Datasets//TRAINING//Happy_test//deamp_6.842601257887742_noise_beach_10_Happy315.wav
D://Datasets//TRAINING//Happy_test//deamp_0.323073062040089_noise_library_12_Happy315.wav
D:/

D://Datasets//TRAINING//Happy_test//deamp_2.3285799367728597_noise_tram_16_Happy169.wav
D://Datasets//TRAINING//Happy_test//deamp_9.325709962413343_noise_office_7_Happy169.wav
D://Datasets//TRAINING//Happy_test//deamp_4.795973330411568_noise_train_15_Happy1387.wav
D://Datasets//TRAINING//Happy_test//deamp_5.51998850153401_noise_city_center_18_Happy1387.wav
D://Datasets//TRAINING//Happy_test//deamp_8.471884830471044_noise_grocery_store_18_Happy955.wav
D://Datasets//TRAINING//Happy_test//deamp_3.0465340019143357_noise_tram_16_Happy955.wav
D://Datasets//TRAINING//Happy_test//deamp_1.9991758538841822_noise_metro_station_15_Happy1103.wav
D://Datasets//TRAINING//Happy_test//deamp_10.572549658046322_noise_forest_path_6_Happy1103.wav
D://Datasets//TRAINING//Happy_test//deamp_3.865493367422373_noise_metro_station_9_Happy904.wav
D://Datasets//TRAINING//Happy_test//deamp_11.548441373155677_noise_residential_area__Happy904.wav
D://Datasets//TRAINING//Happy_test//deamp_7.808972108977907_noise_resid

D://Datasets//TRAINING//Happy_test//deamp_2.2674359669893396_noise_beach_1_Happy1467.wav
D://Datasets//TRAINING//Happy_test//deamp_0.45581950442226304_noise_train_15_Happy1467.wav
D://Datasets//TRAINING//Happy_test//deamp_0.40801323951847923_noise_train_15_Happy268.wav
D://Datasets//TRAINING//Happy_test//deamp_1.0235158465171903_noise_library_13_Happy268.wav
D://Datasets//TRAINING//Happy_test//deamp_4.1066785028354165_noise_office_19_Happy1572.wav
D://Datasets//TRAINING//Happy_test//deamp_1.2277529759139583_noise_grocery_store_3_Happy1572.wav
D://Datasets//TRAINING//Happy_test//deamp_5.957402161747321_noise_home_2_Happy423.wav
D://Datasets//TRAINING//Happy_test//deamp_6.433762497384523_noise_train_5_Happy423.wav
D://Datasets//TRAINING//Happy_test//deamp_11.58279673549719_noise_train_8_Happy136.wav
D://Datasets//TRAINING//Happy_test//deamp_7.315448246209244_noise_grocery_store_17_Happy136.wav
D://Datasets//TRAINING//Happy_test//deamp_2.382038489035455_noise_grocery_store_3_Happy1586.wav

In [85]:
add_noise_and_deamplify_per_folder(a_test, '.wav', background_noise_test)

D://Datasets//TRAINING//Angry_test//deamp_3.9143836969922554_noise_home_12_Angry438.wav
D://Datasets//TRAINING//Angry_test//deamp_2.411686391516828_noise_train_4_Angry438.wav
D://Datasets//TRAINING//Angry_test//deamp_0.09019926958844682_noise_bus_4_Angry239.wav
D://Datasets//TRAINING//Angry_test//deamp_9.305251103545748_noise_grocery_store_17_Angry239.wav
D://Datasets//TRAINING//Angry_test//deamp_11.59690838584974_noise_a130_90_12_Angry974.wav
D://Datasets//TRAINING//Angry_test//deamp_6.993326700658245_noise_office_7_Angry974.wav
D://Datasets//TRAINING//Angry_test//deamp_1.2862597602877819_noise_library_13_Angry589.wav
D://Datasets//TRAINING//Angry_test//deamp_7.459359868993516_noise_grocery_store_6_Angry589.wav
D://Datasets//TRAINING//Angry_test//deamp_6.299556485093654_noise_residential_area_9_Angry484.wav
D://Datasets//TRAINING//Angry_test//deamp_6.503880904651465_noise_forest_path_6_Angry484.wav
D://Datasets//TRAINING//Angry_test//deamp_6.915269448254585_noise_train_15_Angry1380.wa

D://Datasets//TRAINING//Angry_test//deamp_1.9228207216029096_noise_car_6_Angry651.wav
D://Datasets//TRAINING//Angry_test//deamp_5.079916313522215_noise_tram_16_Angry603.wav
D://Datasets//TRAINING//Angry_test//deamp_10.731789970380746_noise_city_center_18_Angry603.wav
D://Datasets//TRAINING//Angry_test//deamp_6.138324787314408_noise_beach_11_Angry85.wav
D://Datasets//TRAINING//Angry_test//deamp_10.34929494639559_noise_park_2_Angry85.wav
D://Datasets//TRAINING//Angry_test//deamp_11.575665180344348_noise_home_2_Angry655.wav
D://Datasets//TRAINING//Angry_test//deamp_1.480604213478752_noise_beach_11_Angry655.wav
D://Datasets//TRAINING//Angry_test//deamp_2.5975484475877533_noise_library_13_Angry616.wav
D://Datasets//TRAINING//Angry_test//deamp_1.5079069315397087_noise_home_12_Angry616.wav
D://Datasets//TRAINING//Angry_test//deamp_5.217924252940913_noise_park_1_Angry820.wav
D://Datasets//TRAINING//Angry_test//deamp_10.721017110775257_noise_a107_30_6_Angry820.wav
D://Datasets//TRAINING//Angry_

D://Datasets//TRAINING//Angry_test//deamp_1.4379235259467156_noise_office_7_Angry1174.wav
D://Datasets//TRAINING//Angry_test//deamp_7.51953952318275_noise_residential_area__Angry1174.wav
D://Datasets//TRAINING//Angry_test//deamp_4.09802091164873_noise_grocery_store_16_Angry1754.wav
D://Datasets//TRAINING//Angry_test//deamp_7.603484182733944_noise_residential_area__Angry1754.wav
D://Datasets//TRAINING//Angry_test//deamp_5.6197954221184965_noise_car_18_Angry1705.wav
D://Datasets//TRAINING//Angry_test//deamp_7.0883749820957_noise_metro_station_5_Angry1705.wav
D://Datasets//TRAINING//Angry_test//deamp_5.031604818024915_noise_metro_station_5_Angry1445.wav
D://Datasets//TRAINING//Angry_test//deamp_11.91250109045054_noise_grocery_store_17_Angry1445.wav
D://Datasets//TRAINING//Angry_test//deamp_9.228340819843126_noise_forest_path_4_Angry644.wav
D://Datasets//TRAINING//Angry_test//deamp_8.35046996231424_noise_home_2_Angry644.wav
D://Datasets//TRAINING//Angry_test//deamp_0.6829192924050069_noise

D://Datasets//TRAINING//Angry_test//deamp_5.149427323293674_noise_park_1_Angry417.wav
D://Datasets//TRAINING//Angry_test//deamp_1.1108430620943945_noise_metro_station_15_Angry851.wav
D://Datasets//TRAINING//Angry_test//deamp_6.297721542901495_noise_park_10_Angry851.wav
D://Datasets//TRAINING//Angry_test//deamp_7.584297246301345_noise_b050_30_6_Angry1096.wav
D://Datasets//TRAINING//Angry_test//deamp_6.969817948418702_noise_park_1_Angry1096.wav
D://Datasets//TRAINING//Angry_test//deamp_3.375545016324735_noise_grocery_store_16_Angry445.wav
D://Datasets//TRAINING//Angry_test//deamp_9.280236059591688_noise_residential_area_9_Angry445.wav
D://Datasets//TRAINING//Angry_test//deamp_10.134290111820569_noise_city_center_8_Angry994.wav
D://Datasets//TRAINING//Angry_test//deamp_0.5960109634714996_noise_beach_11_Angry994.wav
D://Datasets//TRAINING//Angry_test//deamp_10.151400401809678_noise_beach_1_Angry1021.wav
D://Datasets//TRAINING//Angry_test//deamp_5.479245427916048_noise_train_8_Angry1021.wav

D://Datasets//TRAINING//Angry_test//deamp_2.5852796438355456_noise_a107_210_24_Angry5.wav
D://Datasets//TRAINING//Angry_test//deamp_11.209803634429324_noise_office_7_Angry5.wav
D://Datasets//TRAINING//Angry_test//deamp_5.444682490075364_noise_residential_area__Angry468.wav
D://Datasets//TRAINING//Angry_test//deamp_8.114711324590331_noise_train_5_Angry468.wav
D://Datasets//TRAINING//Angry_test//deamp_5.063824974039088_noise_beach_1_Angry752.wav
D://Datasets//TRAINING//Angry_test//deamp_11.775770413244963_noise_park_2_Angry752.wav
D://Datasets//TRAINING//Angry_test//deamp_4.929512398210293_noise_grocery_store_17_Angry547.wav
D://Datasets//TRAINING//Angry_test//deamp_5.115623050873632_noise_metro_station_15_Angry547.wav
D://Datasets//TRAINING//Angry_test//deamp_6.19987231378966_noise_a107_180_21_Angry1101.wav
D://Datasets//TRAINING//Angry_test//deamp_0.9308458189369166_noise_city_center_17_Angry1101.wav
D://Datasets//TRAINING//Angry_test//deamp_6.339545969331439_noise_train_14_Angry246.wa

D://Datasets//TRAINING//Angry_test//deamp_6.800330084177592_noise_residential_area__Angry1268.wav
D://Datasets//TRAINING//Angry_test//deamp_11.181230853276489_noise_library_13_Angry1067.wav
D://Datasets//TRAINING//Angry_test//deamp_6.777659455277376_noise_tram_4_Angry1067.wav
D://Datasets//TRAINING//Angry_test//deamp_2.3469893670943094_noise_car_18_Angry188.wav
D://Datasets//TRAINING//Angry_test//deamp_0.018722509647821628_noise_metro_station_15_Angry188.wav
D://Datasets//TRAINING//Angry_test//deamp_0.9777220716202564_noise_bus_4_Angry1116.wav
D://Datasets//TRAINING//Angry_test//deamp_11.435064970575201_noise_tram_4_Angry1116.wav
D://Datasets//TRAINING//Angry_test//deamp_8.238385234535746_noise_train_4_Angry1093.wav
D://Datasets//TRAINING//Angry_test//deamp_5.0039763255189005_noise_beach_1_Angry1093.wav
D://Datasets//TRAINING//Angry_test//deamp_6.043742562965424_noise_train_15_Angry515.wav
D://Datasets//TRAINING//Angry_test//deamp_6.5317183086351704_noise_beach_1_Angry515.wav
D://Datas

D://Datasets//TRAINING//Angry_test//deamp_2.1451588978059566_noise_city_center_17_Angry1257.wav
D://Datasets//TRAINING//Angry_test//deamp_3.896978247860525_noise_train_4_Angry1257.wav
D://Datasets//TRAINING//Angry_test//deamp_10.49111996043353_noise_city_center_17_Angry1780.wav
D://Datasets//TRAINING//Angry_test//deamp_8.592957568147268_noise_forest_path_3_Angry1780.wav
D://Datasets//TRAINING//Angry_test//deamp_10.304267568520102_noise_train_14_Angry1635.wav
D://Datasets//TRAINING//Angry_test//deamp_6.906241436315463_noise_office_19_Angry1635.wav
D://Datasets//TRAINING//Angry_test//deamp_5.444249963040229_noise_city_center_18_Angry1507.wav
D://Datasets//TRAINING//Angry_test//deamp_1.7137946984089343_noise_residential_area__Angry1507.wav
D://Datasets//TRAINING//Angry_test//deamp_2.5869184641998166_noise_park_1_Angry523.wav
D://Datasets//TRAINING//Angry_test//deamp_6.948772137591326_noise_a062_0_3_Angry523.wav
D://Datasets//TRAINING//Angry_test//deamp_1.9463655703428309_noise_train_4_Ang

D://Datasets//TRAINING//Angry_test//deamp_9.572278120036355_noise_office_7_Angry1790.wav
D://Datasets//TRAINING//Angry_test//deamp_0.8352787957753005_noise_metro_station_15_Angry1214.wav
D://Datasets//TRAINING//Angry_test//deamp_4.076478584784077_noise_residential_area_10_Angry1214.wav
D://Datasets//TRAINING//Angry_test//deamp_9.110407077782096_noise_tram_16_Angry1353.wav
D://Datasets//TRAINING//Angry_test//deamp_6.109488596539457_noise_library_13_Angry1353.wav
D://Datasets//TRAINING//Angry_test//deamp_8.527768976581761_noise_beach_1_Angry475.wav
D://Datasets//TRAINING//Angry_test//deamp_3.494785214404327_noise_train_14_Angry475.wav
D://Datasets//TRAINING//Angry_test//deamp_0.3473356248725037_noise_office_19_Angry87.wav
D://Datasets//TRAINING//Angry_test//deamp_3.7881307428213544_noise_residential_area_10_Angry87.wav
D://Datasets//TRAINING//Angry_test//deamp_5.145894849447851_noise_train_8_Angry532.wav
D://Datasets//TRAINING//Angry_test//deamp_2.23436737519747_noise_city_center_18_Angr

In [86]:
add_noise_and_deamplify_per_folder(n_test, '.wav', background_noise_test)

D://Datasets//TRAINING//Neutral_test//deamp_0.7541876085161765_noise_library_13_Neutral448.wav
D://Datasets//TRAINING//Neutral_test//deamp_2.3659036821233914_noise_grocery_store_3_Neutral448.wav
D://Datasets//TRAINING//Neutral_test//deamp_5.266476187572196_noise_forest_path_3_Neutral1173.wav
D://Datasets//TRAINING//Neutral_test//deamp_7.256577759734137_noise_tram_9_Neutral1173.wav
D://Datasets//TRAINING//Neutral_test//deamp_1.4286550634356163_noise_grocery_store_18_Neutral847.wav
D://Datasets//TRAINING//Neutral_test//deamp_10.371159572310594_noise_residential_area__Neutral847.wav
D://Datasets//TRAINING//Neutral_test//deamp_8.194864820272915_noise_train_14_Neutral1116.wav
D://Datasets//TRAINING//Neutral_test//deamp_7.288875374757678_noise_car_13_Neutral1116.wav
D://Datasets//TRAINING//Neutral_test//deamp_6.946830343383232_noise_car_8_Neutral99.wav
D://Datasets//TRAINING//Neutral_test//deamp_8.833221263134067_noise_forest_path_6_Neutral99.wav
D://Datasets//TRAINING//Neutral_test//deamp_1

D://Datasets//TRAINING//Neutral_test//deamp_11.668434928360092_noise_city_center_18_Neutral466.wav
D://Datasets//TRAINING//Neutral_test//deamp_2.3049247951808924_noise_metro_station_15_Neutral111.wav
D://Datasets//TRAINING//Neutral_test//deamp_10.775430423125744_noise_library_12_Neutral111.wav
D://Datasets//TRAINING//Neutral_test//deamp_2.7683912823071406_noise_beach_11_Neutral1263.wav
D://Datasets//TRAINING//Neutral_test//deamp_3.919191091605077_noise_office_14_Neutral1263.wav
D://Datasets//TRAINING//Neutral_test//deamp_7.953927354150345_noise_residential_area_10_Neutral576.wav
D://Datasets//TRAINING//Neutral_test//deamp_9.426521600837077_noise_train_8_Neutral576.wav
D://Datasets//TRAINING//Neutral_test//deamp_9.633888624319692_noise_forest_path_3_Neutral1064.wav
D://Datasets//TRAINING//Neutral_test//deamp_9.144819805291146_noise_park_1_Neutral1064.wav
D://Datasets//TRAINING//Neutral_test//deamp_5.784661883050954_noise_residential_area__Neutral1102.wav
D://Datasets//TRAINING//Neutral_

D://Datasets//TRAINING//Neutral_test//deamp_8.474162760922992_noise_home_2_Neutral1133.wav
D://Datasets//TRAINING//Neutral_test//deamp_10.640895325322953_noise_car_18_Neutral1303.wav
D://Datasets//TRAINING//Neutral_test//deamp_11.240333005806281_noise_a084_122_15_Neutral1303.wav
D://Datasets//TRAINING//Neutral_test//deamp_1.0800937430877613_noise_office_7_Neutral1082.wav
D://Datasets//TRAINING//Neutral_test//deamp_9.103387111764004_noise_home_12_Neutral1082.wav
D://Datasets//TRAINING//Neutral_test//deamp_6.032123554904323_noise_metro_station_15_Neutral298.wav
D://Datasets//TRAINING//Neutral_test//deamp_4.189882423304191_noise_office_19_Neutral298.wav
D://Datasets//TRAINING//Neutral_test//deamp_1.3521717059826912_noise_park_1_Neutral1478.wav
D://Datasets//TRAINING//Neutral_test//deamp_11.99307968721957_noise_city_center_18_Neutral1478.wav
D://Datasets//TRAINING//Neutral_test//deamp_6.4192324947501955_noise_residential_area__Neutral628.wav
D://Datasets//TRAINING//Neutral_test//deamp_10.0

D://Datasets//TRAINING//Neutral_test//deamp_2.102131391233499_noise_beach_11_Neutral999.wav
D://Datasets//TRAINING//Neutral_test//deamp_2.207311587440884_noise_train_5_Neutral613.wav
D://Datasets//TRAINING//Neutral_test//deamp_9.88658428245441_noise_residential_area_9_Neutral613.wav
D://Datasets//TRAINING//Neutral_test//deamp_6.228326831631639_noise_a107_90_12_Neutral1311.wav
D://Datasets//TRAINING//Neutral_test//deamp_6.06449260757805_noise_grocery_store_3_Neutral1311.wav
D://Datasets//TRAINING//Neutral_test//deamp_1.8232673616438486_noise_a107_30_6_Neutral818.wav
D://Datasets//TRAINING//Neutral_test//deamp_4.592425387734147_noise_city_center_7_Neutral818.wav
D://Datasets//TRAINING//Neutral_test//deamp_2.706241019358665_noise_beach__Neutral748.wav
D://Datasets//TRAINING//Neutral_test//deamp_8.361156800291432_noise_metro_station_5_Neutral748.wav
D://Datasets//TRAINING//Neutral_test//deamp_0.20028443362038129_noise_car_17_Neutral1339.wav
D://Datasets//TRAINING//Neutral_test//deamp_11.04

In [87]:
add_noise_and_deamplify_per_folder(s_test, '.wav', background_noise_test)

D://Datasets//TRAINING//Sad_test//deamp_7.831466711117519_noise_library_14_Sad511.wav
D://Datasets//TRAINING//Sad_test//deamp_5.2987633807168235_noise_tram_16_Sad511.wav
D://Datasets//TRAINING//Sad_test//deamp_5.812004081538621_noise_office_19_Sad1252.wav
D://Datasets//TRAINING//Sad_test//deamp_6.789053828376861_noise_train_5_Sad1252.wav
D://Datasets//TRAINING//Sad_test//deamp_11.878863020815187_noise_library_14_Sad1745.wav
D://Datasets//TRAINING//Sad_test//deamp_8.459125820930721_noise_city_center_17_Sad1745.wav
D://Datasets//TRAINING//Sad_test//deamp_1.5462133868505883_noise_train_4_Sad1093.wav
D://Datasets//TRAINING//Sad_test//deamp_2.054252511261639_noise_beach_11_Sad1093.wav
D://Datasets//TRAINING//Sad_test//deamp_5.985554992685513_noise_tram_16_Sad1192.wav
D://Datasets//TRAINING//Sad_test//deamp_10.164474443724696_noise_train_5_Sad1192.wav
D://Datasets//TRAINING//Sad_test//deamp_6.630044960946979_noise_a084_122_15_Sad1416.wav
D://Datasets//TRAINING//Sad_test//deamp_1.604841633323

D://Datasets//TRAINING//Sad_test//deamp_1.1789561747756525_noise_home_11_Sad261.wav
D://Datasets//TRAINING//Sad_test//deamp_1.977272869373564_noise_office_14_Sad261.wav
D://Datasets//TRAINING//Sad_test//deamp_4.513021939743645_noise_office_19_Sad1153.wav
D://Datasets//TRAINING//Sad_test//deamp_5.916962637201522_noise_bus_4_Sad1153.wav
D://Datasets//TRAINING//Sad_test//deamp_2.6053869364317563_noise_residential_area__Sad1084.wav
D://Datasets//TRAINING//Sad_test//deamp_6.173765215926693_noise_office_14_Sad1084.wav
D://Datasets//TRAINING//Sad_test//deamp_3.149156281763891_noise_a062_0_3_Sad1369.wav
D://Datasets//TRAINING//Sad_test//deamp_11.806685750112353_noise_train_15_Sad1369.wav
D://Datasets//TRAINING//Sad_test//deamp_7.3116980058384335_noise_office_19_Sad1744.wav
D://Datasets//TRAINING//Sad_test//deamp_5.400290160749844_noise_city_center_17_Sad1744.wav
D://Datasets//TRAINING//Sad_test//deamp_2.864647768283569_noise_tram_16_Sad1481.wav
D://Datasets//TRAINING//Sad_test//deamp_7.4484001

D://Datasets//TRAINING//Sad_test//deamp_0.5534694335655868_noise_car_8_Sad1228.wav
D://Datasets//TRAINING//Sad_test//deamp_6.488734913821119_noise_grocery_store_3_Sad1228.wav
D://Datasets//TRAINING//Sad_test//deamp_3.147658197327225_noise_office_19_Sad99.wav
D://Datasets//TRAINING//Sad_test//deamp_5.854546604597903_noise_tram_16_Sad99.wav
D://Datasets//TRAINING//Sad_test//deamp_2.891432612054946_noise_grocery_store_7_Sad1494.wav
D://Datasets//TRAINING//Sad_test//deamp_0.3646834043988405_noise_train_5_Sad1494.wav
D://Datasets//TRAINING//Sad_test//deamp_9.279270262852558_noise_home_2_Sad1268.wav
D://Datasets//TRAINING//Sad_test//deamp_0.6248900295098636_noise_bus_3_Sad1268.wav
D://Datasets//TRAINING//Sad_test//deamp_9.6313762376638_noise_grocery_store_17_Sad489.wav
D://Datasets//TRAINING//Sad_test//deamp_10.172113923858497_noise_a110_120_15_Sad489.wav
D://Datasets//TRAINING//Sad_test//deamp_7.155970425992621_noise_car_8_Sad1212.wav
D://Datasets//TRAINING//Sad_test//deamp_3.43345464033665

D://Datasets//TRAINING//Sad_test//deamp_10.688855765184988_noise_bus_19_Sad1568.wav
D://Datasets//TRAINING//Sad_test//deamp_8.160597363778638_noise_park_1_Sad1568.wav
D://Datasets//TRAINING//Sad_test//deamp_8.781700825556245_noise_a084_122_15_Sad762.wav
D://Datasets//TRAINING//Sad_test//deamp_9.234134032468935_noise_residential_area_9_Sad762.wav
D://Datasets//TRAINING//Sad_test//deamp_11.711232208366324_noise_city_center_18_Sad1327.wav
D://Datasets//TRAINING//Sad_test//deamp_4.648203150434714_noise_car_18_Sad1327.wav
D://Datasets//TRAINING//Sad_test//deamp_1.8471706135424677_noise_beach_10_Sad310.wav
D://Datasets//TRAINING//Sad_test//deamp_7.277383275699404_noise_car_18_Sad310.wav
D://Datasets//TRAINING//Sad_test//deamp_0.958179720254658_noise_bus_19_Sad1629.wav
D://Datasets//TRAINING//Sad_test//deamp_9.963291232886272_noise_b056_90_12_Sad1629.wav
D://Datasets//TRAINING//Sad_test//deamp_6.319490789342469_noise_tram_16_Sad1392.wav
D://Datasets//TRAINING//Sad_test//deamp_3.41733842502570

In [88]:
add_noise_and_deamplify_per_folder(o_test, '.wav', background_noise_test)

D://Datasets//TRAINING//Other_test//deamp_7.073333191750125_noise_office_19_Other1090.wav
D://Datasets//TRAINING//Other_test//deamp_0.44516064017119206_noise_metro_station_9_Other1090.wav
D://Datasets//TRAINING//Other_test//deamp_5.531304198071199_noise_library_13_Other918.wav
D://Datasets//TRAINING//Other_test//deamp_1.1947442850283454_noise_grocery_store_7_Other918.wav
D://Datasets//TRAINING//Other_test//deamp_11.324464135405258_noise_car_18_Other108.wav
D://Datasets//TRAINING//Other_test//deamp_10.81480188182579_noise_library_12_Other108.wav
D://Datasets//TRAINING//Other_test//deamp_4.272066503741373_noise_city_center_7_Other2114.wav
D://Datasets//TRAINING//Other_test//deamp_1.9942172782352308_noise_tram_16_Other2114.wav
D://Datasets//TRAINING//Other_test//deamp_7.654146049197735_noise_train_15_Other2049.wav
D://Datasets//TRAINING//Other_test//deamp_6.0606393341710945_noise_residential_area_10_Other2049.wav
D://Datasets//TRAINING//Other_test//deamp_3.74979801088351_noise_metro_stati

D://Datasets//TRAINING//Other_test//deamp_10.010559265519785_noise_car_5_Other1945.wav
D://Datasets//TRAINING//Other_test//deamp_10.700034241228408_noise_a107_180_21_Other1945.wav
D://Datasets//TRAINING//Other_test//deamp_4.501201930281232_noise_car_8_Other207.wav
D://Datasets//TRAINING//Other_test//deamp_10.513531065221658_noise_train_14_Other207.wav
D://Datasets//TRAINING//Other_test//deamp_11.402582976680634_noise_library_13_Other1740.wav
D://Datasets//TRAINING//Other_test//deamp_4.707834500071863_noise_tram_8_Other1740.wav
D://Datasets//TRAINING//Other_test//deamp_8.026588161818017_noise_tram_9_Other1378.wav
D://Datasets//TRAINING//Other_test//deamp_7.5427457589673805_noise_bus_4_Other1378.wav
D://Datasets//TRAINING//Other_test//deamp_0.6775217139798935_noise_residential_area_9_Other947.wav
D://Datasets//TRAINING//Other_test//deamp_3.193687867996051_noise_beach_1_Other947.wav
D://Datasets//TRAINING//Other_test//deamp_10.082882204614904_noise_residential_area__Other1478.wav
D://Data

D://Datasets//TRAINING//Other_test//deamp_7.166713136150259_noise_train_15_Other656.wav
D://Datasets//TRAINING//Other_test//deamp_11.656653389120796_noise_forest_path_4_Other1972.wav
D://Datasets//TRAINING//Other_test//deamp_10.399574354377563_noise_residential_area_10_Other1972.wav
D://Datasets//TRAINING//Other_test//deamp_1.8338513072112268_noise_a109_150_18_Other2207.wav
D://Datasets//TRAINING//Other_test//deamp_9.694528831382986_noise_metro_station_9_Other2207.wav
D://Datasets//TRAINING//Other_test//deamp_3.3269952881813127_noise_library_12_Other684.wav
D://Datasets//TRAINING//Other_test//deamp_11.411120279460354_noise_beach_11_Other684.wav
D://Datasets//TRAINING//Other_test//deamp_8.029718343103507_noise_tram_9_Other709.wav
D://Datasets//TRAINING//Other_test//deamp_1.3758525251704934_noise_tram_8_Other709.wav
D://Datasets//TRAINING//Other_test//deamp_7.875242497142753_noise_city_center_17_Other1479.wav
D://Datasets//TRAINING//Other_test//deamp_0.2707536569504603_noise_train_5_Othe

D://Datasets//TRAINING//Other_test//deamp_9.990971054775486_noise_grocery_store_16_Other2085.wav
D://Datasets//TRAINING//Other_test//deamp_9.66045305785715_noise_tram_8_Other2085.wav
D://Datasets//TRAINING//Other_test//deamp_0.35749787115504095_noise_grocery_store_3_Other2313.wav
D://Datasets//TRAINING//Other_test//deamp_1.8092628866393325_noise_bus_3_Other2313.wav
D://Datasets//TRAINING//Other_test//deamp_9.162807762894234_noise_city_center_18_Other123.wav
D://Datasets//TRAINING//Other_test//deamp_1.5863837751248804_noise_train_15_Other123.wav
D://Datasets//TRAINING//Other_test//deamp_8.872941369870302_noise_b056_90_12_Other79.wav
D://Datasets//TRAINING//Other_test//deamp_10.228153463259838_noise_library_11_Other79.wav
D://Datasets//TRAINING//Other_test//deamp_1.5789938091708913_noise_library_13_Other632.wav
D://Datasets//TRAINING//Other_test//deamp_2.854679349348292_noise_forest_path_3_Other632.wav
D://Datasets//TRAINING//Other_test//deamp_7.022744748185271_noise_b056_90_12_Other1725

D://Datasets//TRAINING//Other_test//deamp_7.925577888949483_noise_train_14_Other1534.wav
D://Datasets//TRAINING//Other_test//deamp_4.916317235358489_noise_home_11_Other1534.wav
D://Datasets//TRAINING//Other_test//deamp_8.491334323762228_noise_park_1_Other545.wav
D://Datasets//TRAINING//Other_test//deamp_0.4132163882855644_noise_train_8_Other545.wav
D://Datasets//TRAINING//Other_test//deamp_0.0814571828691335_noise_forest_path_6_Other460.wav
D://Datasets//TRAINING//Other_test//deamp_0.26764655186762587_noise_grocery_store_17_Other460.wav
D://Datasets//TRAINING//Other_test//deamp_7.202016502766835_noise_beach_11_Other1335.wav
D://Datasets//TRAINING//Other_test//deamp_1.5068226252899528_noise_a084_122_15_Other1335.wav
D://Datasets//TRAINING//Other_test//deamp_9.205039168465419_noise_city_center_18_Other511.wav
D://Datasets//TRAINING//Other_test//deamp_9.340087992050938_noise_office_7_Other511.wav
D://Datasets//TRAINING//Other_test//deamp_1.6944667898171368_noise_car_18_Other855.wav
D://Da

...

D://Datasets//TRAINING//Other_test//deamp_7.338326459899851_noise_home_11_Other1669.wav
D://Datasets//TRAINING//Other_test//deamp_10.986928606798145_noise_beach_1_Other1636.wav
D://Datasets//TRAINING//Other_test//deamp_7.396961473156885_noise_train_5_Other1636.wav
D://Datasets//TRAINING//Other_test//deamp_2.1987026843211783_noise_grocery_store_17_Other134.wav
D://Datasets//TRAINING//Other_test//deamp_8.858737277865025_noise_train_15_Other134.wav
D://Datasets//TRAINING//Other_test//deamp_9.306928310294467_noise_office_7_Other1128.wav
D://Datasets//TRAINING//Other_test//deamp_1.4145804751353137_noise_b052_0_3_Other1128.wav
D://Datasets//TRAINING//Other_test//deamp_5.770862349258277_noise_grocery_store_16_Other2257.wav
D://Datasets//TRAINING//Other_test//deamp_6.728150151829759_noise_bus_18_Other2257.wav
D://Datasets//TRAINING//Other_test//deamp_0.09182959641861244_noise_beach_1_Other2050.wav
D://Datasets//TRAINING//Other_test//deamp_1.9155029754993445_noise_train_14_Other2050.wav
D://Dat

D://Datasets//TRAINING//Other_test//deamp_5.397815174315645_noise_home_2_Other191.wav
D://Datasets//TRAINING//Other_test//deamp_7.367595896420841_noise_residential_area_10_Other1280.wav
D://Datasets//TRAINING//Other_test//deamp_11.47369366020514_noise_residential_area_10_Other1280.wav
D://Datasets//TRAINING//Other_test//deamp_0.26599218423281457_noise_a130_90_12_Other1860.wav
D://Datasets//TRAINING//Other_test//deamp_0.357102962504539_noise_residential_area__Other1860.wav
D://Datasets//TRAINING//Other_test//deamp_1.550662390828366_noise_office_7_Other626.wav
D://Datasets//TRAINING//Other_test//deamp_10.580676967488968_noise_home_3_Other626.wav
D://Datasets//TRAINING//Other_test//deamp_0.24019094860310952_noise_metro_station_15_Other2069.wav
D://Datasets//TRAINING//Other_test//deamp_8.933044090108435_noise_car_5_Other2069.wav
D://Datasets//TRAINING//Other_test//deamp_7.711380065241807_noise_train_15_Other1661.wav
D://Datasets//TRAINING//Other_test//deamp_4.799913059609878_noise_train_15

D://Datasets//TRAINING//Other_test//deamp_0.9058657554078775_noise_beach_10_Other1334.wav
D://Datasets//TRAINING//Other_test//deamp_9.392561743588086_noise_office_14_Other58.wav
D://Datasets//TRAINING//Other_test//deamp_0.022181356059462143_noise_metro_station_5_Other58.wav
D://Datasets//TRAINING//Other_test//deamp_8.616690863084331_noise_tram_4_Other1504.wav
D://Datasets//TRAINING//Other_test//deamp_7.803208049065261_noise_train_14_Other1504.wav
D://Datasets//TRAINING//Other_test//deamp_10.023847934844646_noise_home_2_Other2180.wav
D://Datasets//TRAINING//Other_test//deamp_7.634185843370086_noise_office_7_Other2180.wav
D://Datasets//TRAINING//Other_test//deamp_3.938083828444974_noise_beach_1_Other1309.wav
D://Datasets//TRAINING//Other_test//deamp_0.9683051696986058_noise_park_1_Other1309.wav
D://Datasets//TRAINING//Other_test//deamp_7.189267634297217_noise_train_5_Other1509.wav
D://Datasets//TRAINING//Other_test//deamp_11.278779581857831_noise_home_12_Other1509.wav
D://Datasets//TRAIN

D://Datasets//TRAINING//Other_test//deamp_9.812389610430449_noise_grocery_store_3_Other2483.wav
D://Datasets//TRAINING//Other_test//deamp_9.139372110745013_noise_residential_area_10_Other295.wav
D://Datasets//TRAINING//Other_test//deamp_3.7639387091459797_noise_library_11_Other295.wav
D://Datasets//TRAINING//Other_test//deamp_1.9731215856988622_noise_grocery_store_3_Other468.wav
D://Datasets//TRAINING//Other_test//deamp_7.448246766823688_noise_train_8_Other468.wav
D://Datasets//TRAINING//Other_test//deamp_8.665073723291702_noise_car_18_Other379.wav
D://Datasets//TRAINING//Other_test//deamp_9.202037624857176_noise_park_1_Other379.wav
D://Datasets//TRAINING//Other_test//deamp_6.595817439108898_noise_park_11_Other313.wav
D://Datasets//TRAINING//Other_test//deamp_1.2822901739615018_noise_a130_90_12_Other313.wav
D://Datasets//TRAINING//Other_test//deamp_11.34552770958659_noise_train_8_Other1848.wav
D://Datasets//TRAINING//Other_test//deamp_0.9642682824644404_noise_park_1_Other1848.wav
D://D

D://Datasets//TRAINING//Other_test//deamp_5.070124290405563_noise_residential_area__Other385.wav
D://Datasets//TRAINING//Other_test//deamp_6.036298876647962_noise_home_11_Other385.wav
D://Datasets//TRAINING//Other_test//deamp_9.644054535999242_noise_city_center_8_Other2529.wav
D://Datasets//TRAINING//Other_test//deamp_0.19860181719453163_noise_tram_4_Other2529.wav
D://Datasets//TRAINING//Other_test//deamp_1.832744776766552_noise_office_7_Other1525.wav
D://Datasets//TRAINING//Other_test//deamp_10.89759134800094_noise_grocery_store_16_Other1525.wav
D://Datasets//TRAINING//Other_test//deamp_7.904423572048487_noise_grocery_store_16_Other23.wav
D://Datasets//TRAINING//Other_test//deamp_2.0242477761846613_noise_beach_1_Other23.wav
D://Datasets//TRAINING//Other_test//deamp_6.915050180937886_noise_grocery_store_7_Other1053.wav
D://Datasets//TRAINING//Other_test//deamp_6.28873848770168_noise_park_11_Other1053.wav
D://Datasets//TRAINING//Other_test//deamp_10.025260060557237_noise_city_center_18_

In [ ]:
# AudioEffectsChain().reverb()(infile, outfile)